# このページについて
このページでは、Distributionally Robust Optimization(https://arxiv.org/abs/2411.02549)の第1章から2章の勉強内容をまとめていこうと思います。


## 分布ロバスト最適化とは
最適化問題において、確実に既知であるようなパラメータをつかって最適化を行うケースは、実世界での意思決定ではあまり考えられません。それに対して、不確実性を考慮しながら最適化を行うという考えが生まれました。最もナイーブなやり方は、その不確実なパラメータを確率変数と見立ててその期待値を取って演算をすることです。しかし、実際に取る値は平均と大きく離れる場合があります(Savage, Scholtes and Zweidler 2006, Savage 2012)。平均値から大きく離れることを避けるために、以下のような**確率的計画法**が導入されました。
$$
\begin{equation*}
\inf _{x \in \mathcal{X}} \mathbb{E}_{\mathbb{P}}[\ell(x, Z)] \tag{1.1}
\end{equation*}
$$
これは、不確実なパラメータの期待値を直接取るのではなく、$\ell$に対して期待値を取ります。$\ell$は確率変数を入力とする関数ですが、確率変数とみなすことができます。これはいい感じの定式化なのですが、**確率分布が既知**であるという仮定が必要です。さらに、この定式化は次元の呪いがあります。そこで以下のように新しい定式化を考えます。
$$
\inf _{x \in \mathcal{X}} \sup _{z \in \mathcal{Z}} \ell(x, z)
$$
この定式化を、**ロバスト最適化問題**といいます。例えば、$\ell$が状態価値関数であれば、ロバストMDPにおける強化学習と考えるができます。しかし、ロバスト最適化は、目的関数の実現値を直接的に考慮して最適化をするため、過度に保守的になる場合があります。それに対して、目的関数の期待値に対してロバスト最適化問題を解く方法を**分布ロバスト最適化**問題といいます。
$$
\begin{equation*}
\inf _{x \in \mathcal{X}} \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(x, Z)] \tag{1.2}
\end{equation*}
$$


## 不確実性集合
ロバストMDPで主に使われる不確実性集合はKL集合や$L_2$ノルムの集合などが挙げられます。ここでは、より一般化された分布ロバスト最適化という枠組みで、不確実性集合をまとめていこうと思います。

### モーメント不確実性集合
| 不確実性集合の型                            | 定義                                                                                                                                                                                                                              | 特徴と性質                                            | 直感？                                                                                                    |
| ----------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------ | ------------------------------------------------------------------------------------------------------ |
| **2.1.1. サポートのみの不確実性集合**            | $\mathcal{P} = \mathcal{P}(\mathcal{Z})$ (つまり、$\mathcal{Z}$ 上でサポートされるすべての確率分布の集合)                                                                                                                                               | 最も保守的 (範囲内のすべての可能性を考慮),計算が比較的容易                  | 不確実なパラメータが取りうる値の範囲（サポート）のみが分かっている状況を表す。範囲内のすべての分布を考慮することで、最悪の場合のリスクを評価し、ロバストな意思決定を行う。古典的なロバスト最適化に対応する。 |
| **2.1.2. マルコフ不確実性集合**               | $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}[Z]=\mu\right\}$ (平均が $\mu$ である $\mathcal{Z}$ 上のすべての分布)                                                                                     | 平均に関する情報のみを使用,計算が容易                              | $\mu$ のみが分かっている場合、確率変数が特定の閾値を超える確率の上界をマルコフの不等式で評価する。ロバストなリスク評価に利用される。                                  |
| **2.1.3. チェビシェフ不確実性集合**             | $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}[Z]=\mu, \mathbb{E}_{\mathbb{P}}\left[Z Z^{\top}\right]=M\right\}$ (平均が $\mu$、2 次モーメントが $M$ である $\mathcal{Z}$ 上のすべての分布)                     | 平均と分散の情報を使用,計算の複雑さが増す可能性                         | 平均と分散が分かっている場合に、チェビシェフの不等式を用いて、確率変数が平均からどれだけ離れるかの確率に上界を与える。分布の形状に関する情報が少ない状況でも、ある程度の保証を得ることができる。       |
| **2.1.4. 不確実なモーメントを持つチェビシェフ不確実性集合** | $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}):\left(\mathbb{E}_{\mathbb{P}}[Z], \mathbb{E}_{\mathbb{P}}\left[Z Z^{\top}\right]\right) \in \mathcal{F}\right\}$ (モーメントが $\mathcal{F}$ に属する $\mathcal{Z}$ 上のすべての分布) | モーメントの推定誤差を考慮, $\mathcal{F}$ の形状によって不確実性を表現している。 | 平均と分散の推定値に誤差がある場合、その誤差を考慮に入れる。 $\mathcal{F}$ は、平均と分散の可能な値の範囲を表し、モーメントの不確実性が表現される。                      |
| **2.1.5. 平均-分散不確実性集合**              | $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}[Z]=\mu, \mathbb{E}_{\mathbb{P}}[G(Z)] \leq \mathcal{K} g\right\}$ (平均と「分散」に関する制約を持つ分布)                                                     | 多様な分布の特性をモデル化可能, リフトされた集合 $\mathcal{Q}$ との関係     | 平均と、分布の広がりを表す指標（分散の一般化）に関する制約を用いる。平均と分散だけでなく、他の形状に関する情報を表現できる。                                         |
| **2.1.6. 高次モーメント不確実性集合**            | $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}\left[Z^{\alpha}\right] \leq m_{\alpha} \forall \alpha \in \mathcal{A}\right\}$ (高次モーメントに関する制約を持つ分布)                                        | 分布の形状をより詳細に表現,一般的に計算が複雑,NP 困難な場合がある              | より詳しく分布の形状を表現するために、高次のモーメント（歪度、尖度など）に関する制約を用いる。                                                        |

*   **$\mathbb{E}_{\mathbb{P}}[f(Z)]$**：確率分布 $\mathbb{P}$ に関するモーメント関数 $f$ の期待値
*   **$\mathcal{F}$**：モーメント不確実性集合。モーメントの可能な値の範囲
*   **$\mathcal{Z}$**：確率変数 $Z$ の取りうる値の集合（サポート）



### $\phi$-ダイバージェンス不確実性集合
| 不確実性集合の型                           | 定義                                                                                                                                                                                                                                                                                                                                   | 特徴と性質                                                                                          | 数式の裏にある直感                                                                                              |
| ---------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ | ---------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------ |
| **2.2. $\phi$-ダイバージェンス不確実性集合**     | $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{D}_{\phi}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}$ (参照分布 $\hat{\mathbb{P}}$ からの $\phi$-ダイバージェンスが $r$ 以内の分布の集合)                                                                                                                                             | *   汎用性: 様々な種類の非類似性を表現可能  *   $\phi$ の選択によって、様々な分布の性質を捉えられる  *   双対表現を持ち、計算がしやすい場合がある          | 2つの確率分布間の「距離」を測る $\phi$-ダイバージェンスを用いて、参照分布からのずれを制限する。 $\phi$ の選択により、KLダイバージェンス、全変動距離など、様々な種類の距離を表現できる。  |
| **2.2.1. KL不確実性集合**                | $\mathcal{P}=\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}$ (参照分布 $\hat{\mathbb{P}}$ からの KLダイバージェンスが $r$ 以内の分布の集合)                                                                                                                                                             | *   絶対連続性の保証: $\mathbb{P} \ll \hat{\mathbb{P}}$ である分布のみを含む  *   エントロピー的リスクとの関連性  *   統計的学習への応用 | KLダイバージェンスを用いて、参照分布からの「情報量」のずれを制限する。これは、尤度比の対数に対応し、より多くの情報を共有する分布ほど距離が近くなる。                            |
| **2.2.2. 尤度不確実性集合**                | $\mathcal{P}=\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{KL}(\hat{\mathbb{P}}, \mathbb{P}) \leq r\}$ ($\mathbb{P}$ から見た $\hat{\mathbb{P}}$ の KLダイバージェンスが $r$ 以内の分布の集合)                                                                                                                                                  | *   経験的尤度との関連性  *   統計的最適性  *   特に離散分布に対して有効                                                   | 参照分布を $\mathbb{P}$ から見た KLダイバージェンスで制限する。経験分布に基づいて、データに対する尤度がある程度以上ある分布を考慮する。                           |
| **2.2.3. 全変動不確実性集合**               | $\mathcal{P}=\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{TV}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}$ (全変動距離が $r$ 以内の分布の集合)                                                                                                                                                                                                    | *   確率の違いを直接的に表現  *   Huber の汚染モデルとの関連性  *   ロバスト統計への応用                                        | 2つの分布間の確率の差の最大値を制限する。これは、2つの分布がどれだけ「違うか」を直接的に表現する。 Huber の汚染モデルのように、データに外れ値が含まれる場合に、ロバストな意思決定を行うのに役立つ。 |
| **2.2.4. $\chi^2$-ダイバージェンス不確実性集合** | $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \chi^{2}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}$ (ピアソン $\chi^2$-ダイバージェンスが $r$ 以内の分布の集合) <br> または <br>  $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \chi^{2}(\hat{\mathbb{P}}, \mathbb{P}) \leq r\right\}$ (ネイマン $\chi^2$-ダイバージェンスが $r$ 以内の分布の集合) | *   ピアソン $\chi^2$ またはネイマン $\chi^2$ ダイバージェンスを使用  *   統計的検定との関連性                                 | ピアソン $\chi^2$ またはネイマン $\chi^2$ ダイバージェンスを用いて、参照分布とのずれを制限する。これは、分布間の「適合度」を測る。                            |

*   **$\phi$**：エントロピー関数
*   **$\phi^{\infty}(1)$**：$\phi$ のリセッション関数
*   **$\mathrm{D}_{\phi}(\mathbb{P}, \hat{\mathbb{P}})$**：$\phi$-ダイバージェンス
*   **$\operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}})$**：KLダイバージェンス
*   **$\mathrm{TV}(\mathbb{P}, \hat{\mathbb{P}})$**：全変動距離
*   **$\chi^{2}(\mathbb{P}, \hat{\mathbb{P}})$**：$\chi^2$-ダイバージェンス
*   **$\mathbb{P} \ll \hat{\mathbb{P}}$**：$\mathbb{P}$ が $\hat{\mathbb{P}}$ に関して絶対連続
*   **$\mathcal{F}$**：すべての有界ボレル関数 $f: \mathcal{Z} \rightarrow \operatorname{dom}\left(\phi^{*}\right)$ の族
### 最適輸送不確実性集合
TODO